In [ ]:
import os
import time
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ──────────────────────────────────────────────────────────────────────────────
# 1) .env 로드 및 환경변수 검증
# ──────────────────────────────────────────────────────────────────────────────
load_dotenv()  # .env 파일 읽어들여 os.environ에 로드

NAVER_ID = os.getenv("NAVER_ID")
NAVER_PW = os.getenv("NAVER_PW")
if not NAVER_ID or not NAVER_PW:
    raise EnvironmentError(
        "환경변수 NAVER_ID, NAVER_PW를 설정해주세요.\n"
        "프로젝트 루트에 .env 파일을 만들고 NAVER_ID / NAVER_PW를 채워넣으시면 됩니다."
    )

# ──────────────────────────────────────────────────────────────────────────────
# 2) Chrome WebDriver & WebDriverWait 초기화
# ──────────────────────────────────────────────────────────────────────────────
chrome_opts = Options()
chrome_opts.add_argument("--start-maximized")
# headless 로 돌리고 싶으면 아래 주석 해제
# chrome_opts.add_argument("--headless")
# chrome_opts.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=chrome_opts)
wait = WebDriverWait(driver, 10)

try:
    # ──────────────────────────────────────────────────────────────────────────
    # 3) 네이버 로그인
    # ──────────────────────────────────────────────────────────────────────────
    driver.get("https://nid.naver.com/nidlogin.login")
    # ID/PW 입력 필드가 로드될 때까지 대기
    wait.until(EC.presence_of_element_located((By.ID, "id")))
    driver.find_element(By.ID, "id").send_keys(NAVER_ID)
    driver.find_element(By.ID, "pw").send_keys(NAVER_PW)
    driver.find_element(By.ID, "log.login").click()

    # 로그인 성공 후 뜨는 요소(예: 네이버 메인 검색창)까지 대기
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#NM_REALTIME_SEARCH")))

    # ──────────────────────────────────────────────────────────────────────────
    # 4) 카페 게시글 페이지로 이동
    # ──────────────────────────────────────────────────────────────────────────
    cafe_url = (
        "https://cafe.naver.com/f-e/cafes/19997929/articles/90723"
        "?menuid=809&referrerAllArticles=false&inCafeSearch=true&query=시험"
    )
    driver.get(cafe_url)

    # 메인 프레임이 준비될 때까지 대기 후 전환
    wait.until(EC.frame_to_be_available_and_switch_to_it(("cafe_main",)))

    # ──────────────────────────────────────────────────────────────────────────
    # 5) 댓글 중 첫 번째 내용 크롤링
    # ──────────────────────────────────────────────────────────────────────────
    comment_selector = ".u_cbox_comment_box .u_cbox_contents"
    first_comment = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, comment_selector)))
    print("첫 댓글 내용:", first_comment.text)

    # ──────────────────────────────────────────────────────────────────────────
    # 6) 필요에 따라 추가 크롤링 로직 작성...
    # ──────────────────────────────────────────────────────────────────────────

finally:
    time.sleep(1)
    driver.quit()
